<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/images/clemson_paw.png"> </div>


## Homework 6: Logistic Regression

**Clemson University**<br>
**Fall 2021**<br>
**Instructor(s):** Nina Hubig <br>


---



In [ ]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)

### INSTRUCTIONS

- To submit your assignment, follow the instructions given in Canvas.
- Restart the kernel and run the whole notebook again before you submit. 
- As much as possible, try and stick to the hints and functions we import at the top of the homework, as those are the ideas and tools the class supports and are aiming to teach. And if a problem specifies a particular library, you're required to use that library, and possibly others from the import list.
- Please use .head() when viewing data. Do not submit a notebook that is excessively long because output was not suppressed or otherwise limited. 

In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from scipy.stats import ttest_ind

<div class='theme'> Cancer Classification from Gene Expressions </div>

In this problem, we will build a classification model to distinguish between two related classes of cancer, acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML), using gene expression measurements. The dataset is provided in the file `hw6_enhance.csv`. Each row in this file corresponds to a tumor tissue sample from a patient with one of the two forms of Leukemia. The first column contains the cancer type, with **0 indicating the ALL** class and **1 indicating the AML** class. Columns 2-7130 contain expression levels of 7129 genes recorded from each tissue sample. 

In the following questions, we will use linear and logistic regression to build classification models for this data set. 


<div class='exercise'><b> Question 1 [20 pts]: Data Exploration </b></div>

The first step is to split the observations into an approximate 80-20 train-test split.  Below is some code to do this for you (we want to make sure everyone has the same splits). Print dataset shape before splitting and after splitting. `Cancer_type` is our target column.

In [ ]:
""" 
Use this code to load and split your data set in order to ensure that everyone
has the same splits. 
"""

# Load data
df = pd.read_csv('hw6_enhance.csv', index_col=0)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Cancer_type'], 
                                                    df.Cancer_type, 
                                                    test_size=0.2, 
                                                    random_state=4300, 
                                                    stratify=df.Cancer_type)

print(df.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(df.Cancer_type.value_counts(normalize=True))

**1.1** Take a peek at your training set (Using the `describe()` method): you should notice the severe differences in the measurements from one gene to the next (some are negative, some hover around zero, and some are well into the thousands). To account for these differences in scale and variability, normalize each predictor to vary between 0 and 1. **NOTE: for the entirety of this homework assignment, you will use these normalized values, not the original, raw values**. To make the provided code in the remainder of the notebook work without modification, name the scaled train and test sets `X_train` and `X_test`. 

In [ ]:
# Your code here

**1.2** The training set contains more predictors than observations. What problem(s) can this lead to in fitting a classification model to such a dataset? Explain in 3 or fewer sentences.

*Your answer here*

**1.3** Next we want to determine which 10 genes individually discriminate between the two cancer classes the best (consider every gene in the dataset). Code has been provided to do this for you. Make sure you understand what the code is doing. Note that it makes use of [t-testing](https://en.wikipedia.org/wiki/Welch%27s_t-test).

Plot two histograms of best predictor -- one using the training set and another using the testing set. Each histogram should clearly distinguish two different `Cancer_type` classes.

In [ ]:
"""
This code uses t-values to determine which genes discriminate between the two
cancer classes the best. 
"""
predictors = df.columns
predictors = predictors.drop('Cancer_type');
print(predictors.shape) 

means_0 = X_train[y_train==0][predictors].mean()
means_1 = X_train[y_train==1][predictors].mean()
stds_0 = X_train[y_train==0][predictors].std()
stds_1 = X_train[y_train==1][predictors].std()
n1 = X_train[y_train==0].shape[0]
n2 = X_train[y_train==1].shape[0]

t_tests = np.abs(means_0-means_1)/np.sqrt( stds_0**2/n1 + stds_1**2/n2)

# Your code here
best_preds_idx = np.argsort(-t_tests.values)
best_preds = t_tests.index[best_preds_idx]

print(t_tests[best_preds_idx[0:10]])
print(t_tests.index[best_preds_idx[0:10]])

best_pred = t_tests.index[best_preds_idx[0]]
print(best_pred)

In [ ]:
# Your code here

**1.4** Using your most useful gene from the previous part, create a classification model by simply eye-balling a value for this gene that would discriminate the two classes the best (do not use an algorithm to determine for you the optimal coefficient or threshold; we are asking you to provide a rough estimate / model by manual inspection). Justify your choice in 1-2 sentences. Report the accuracy of your hand-chosen model on the test set (write code to implement and evaluate your hand-created model).

In [ ]:
# Your code here

*Your answer here*

<hr>

<div class='exercise'><b> Question 2 [25 pts]: Linear and Logistic Regression  </b></div>

In class, we discussed how to use both linear regression and logistic regression for classification. For this question, you will explore these two models by working with the single gene that you identified above as being the best predictor.

**2.1** Fit a simple linear regression model to the training set using the single gene predictor "best_predictor" to predict cancer type (use the normalized values of the gene). We could interpret the scores predicted by the regression model for a patient as being an estimate of the probability that the patient has Cancer_type=1 (AML). Is this a reasonable interpretation? If not, what is the problem with such?

Create a figure with the following items displayed on the same plot (Use training data):
 - the model's predicted value (the quantitative response from your linear regression model as a function of the normalized value of the best gene predictor)
 - the true binary response. 

In [ ]:
# Your code here

*Your answer here*

**2.2** Use your estimated linear regression model to classify observations into 0 and 1 using the standard Bayes classifier, which uses a threshold of 0.5. Evaluate the classification accuracy of this classification model on both the training and testing sets.

In [ ]:
# Your code here

**2.3** Next, fit a simple logistic regression model to the training set. How do the training and test classification accuracies of this model compare with the linear regression model? 

Remember, you need to set the regularization parameter for sklearn's logistic regression function to be a very large value in order to **not** regularize (use 'C=100000'). 

In [ ]:
# Your code here

*Your answer here* 

**2.4** Create 2 plots (one with training and one with test data) with 4 items displayed on each plot.
- the quantitative response from the linear regression model as a function of the best gene predictor.
- the predicted probabilities of the logistic regression model as a function of the best gene predictor. 
- the true binary response. 
- a horizontal line at $y=0.5$. 

Based on these plots, does one of the models appear better suited for binary classification than the other?  Explain in 3 sentences or fewer. 

In [ ]:
# Your code here

*Your answer here*

<hr>

<div class='exercise'> <b> Question 3 [20pts]: Multiple Logistic Regression </b> </div>

**3.1** Next, fit a multiple logistic regression model with **all** the gene predictors from the data set (reminder: for this assignment, we are always using the normalized values). How does the classification accuracy of this model compare with the models fitted in question 2 with a single gene (on both the training and test sets)?  

In [ ]:
# Your code here

*Your answer here* 

**3.2** Comment on the classification accuracy of both the training and testing set. Given the results above, how would you assess the generalization capacity of your trained model? What other tests would you suggest to better guard against possibly having a false sense of the overall efficacy/accuracy of the model as a whole?

*Your answer here*

**3.3** Now let's use regularization to improve the predictions from the multiple logistic regression model. Specifically, use LASSO-like regularization (i.e., L1 penalty) and 5-fold cross-validation to train the model on the training set. Report the classification accuracy on both the training and testing set.

In [ ]:
# Your code here

**3.4** Which model do you think generalizes the best? Explain your answer. 

*Your answer here*